# 使用自己训练的模型权重进行推理，并输出PSNR和SSIM

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# 加载SRCNN模型
def load_srcnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (9, 9), activation='relu', padding='same', input_shape=(None, None, 3)),
        tf.keras.layers.Conv2D(32, (1, 1), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(3, (5, 5), padding='same')
    ])
    return model




# 加载模型权重

In [ ]:

def load_model_weights(model, weights_path):
    model.load_weights(weights_path)
    return model


# 超分辨率推理

In [ ]:

def super_resolution(model, image):
    image = image / 255.0  # 归一化
    image = np.expand_dims(image, axis=0)  # 扩展维度，变为4维张量
    sr_image = model.predict(image)  # 预测超分辨率图像
    sr_image = np.clip(sr_image, 0, 1) * 255.0  # 将图像像素值截断到 [0, 255] 范围内
    sr_image = np.squeeze(sr_image, axis=0)  # 去除扩展的维度
    return sr_image.astype(np.uint8)



# 计算PSNR和SSIM指标值

In [ ]:

def calculate_metrics(sr_image, hr_image):
    psnr_value = peak_signal_noise_ratio(hr_image, sr_image,data_range=255)
    ssim_value = structural_similarity(hr_image, sr_image, multichannel=True,data_range=255,channel_axis=2)
    return psnr_value, ssim_value



# 加载自己刚刚训练好的模型

In [ ]:

print("Loading SRCNN model...")
srcnn_model = load_srcnn_model()
srcnn_model = load_model_weights(srcnn_model, "srcnn.weights.h5")



# 读取测试图像（输入自己的图片）

In [ ]:

print("Loading test image...")
test_image_path = "123.png"
test_image = cv2.imread(test_image_path)



In [ ]:
# 超分辨率推理
print("Upscaling test image...")
sr_image = super_resolution(srcnn_model, test_image)



In [ ]:
# 将hr_image调整为与sr_image相同大小
hr_image = cv2.resize(test_image, (sr_image.shape[1], sr_image.shape[0]), interpolation=cv2.INTER_CUBIC)



# 计算PSNR和SSIM指标值

In [ ]:

print("Calculating metrics...")
psnr_value, ssim_value = calculate_metrics(sr_image, hr_image)
print(f"\nPSNR: {psnr_value:.2f} dB")
print(f"SSIM: {ssim_value:.4f}")
